# НИС. Лабораторная работа 1. 
### Мельников Евгений 18ПМИ

## Установка зависимостей

In [ ]:
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
!python -m pip install nemo_toolkit[all]

# !pip install torchtext==0.10.1
# !pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""

! pip install python-Levenshtein

! rm -rf sample_data
# exit()

In [2]:
from omegaconf import DictConfig

In [3]:
from Levenshtein import distance as lv_dist
from Levenshtein import ratio as lv_ratio
lv_dist("hello", "hedf"), lv_ratio("hello", "hedf")

(3, 0.4444444444444444)

## Загрузка тренировочного датасета на колаб машину и подготовка данных



Записи голосов для тренировки будут загружаться на колаб машину и распаковываться в папку /content/farfield_data/train на ней же

Манифесты для тренировки находятся в гугл драйве в папке /drive/MyDrive/nis_data/farfield_train

Записи голосов для тестирования, а также манифесты находятся в папке /drive/MyDrive/nis_data/farfield_test

In [11]:
# mounting the google drive
from google.colab import drive
import os
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [12]:
print(os.listdir('/drive/MyDrive/nis_data'))
print(os.listdir('/drive/MyDrive/nis_data/farfield_test'))
print(os.listdir('/drive/MyDrive/nis_data/farfield_train'))

['farfield_test', 'farfield_train']
['files', 'manifest_abs.jsonl', 'manifest.jsonl']
['files', 'manifest_abs.jsonl', 'manifest.jsonl', 'manifest_all.jsonl']


In [14]:
# Очистка файла с манифестами для тренировки
import json
def clear_json(path, filename_in, filename_out):
  path_in = os.path.join(path, filename_in)
  path_out = os.path.join(path, filename_out)
  with open(path_in, 'r') as fin:
    with open(path_out, 'w+') as fout:
      for line in fin:
        metadata = json.loads(line)
        if metadata['audio_filepath'].startswith("farfield"):
          json.dump(metadata, fout)
          fout.write('\n')

# clear_json("/drive/MyDrive/nis_data/farfield_train/", "manifest_all.jsonl", "manifest.jsonl")

In [6]:
# Папка, где будет размещаться датасет Golos
data_dir = '/content'
tar_dir = 'tar' # directory where all .tar files are stored 

test_data_abs_path = '/drive/MyDrive/nis_data/farfield_test/files'
train_data_abs_path = '/content/farfield_data/train/farfield'

### Загружаем и распаковываем архив с записями для тренировки в колаб машине



In [8]:
import glob
import os
import subprocess
import tarfile
import wget
from shutil import copyfile

def load(url, fname, path):
  if not os.path.exists(os.path.join(path, fname)):
    print(f"Downloading {fname}...")
    out_path = wget.download(url, os.path.join(path, fname))
    print(f"Downloaded at: {out_path}")
  else:
    print("File already exists")
    out_path = os.path.join(path, fname)
  return out_path

if not os.path.exists(os.path.join(data_dir, tar_dir)):
  os.mkdir(tar_dir) # directory where all .tar files are stored

train_tar_path = load("https://sc.link/1Z3", "farfield.tar", os.path.join(data_dir, tar_dir)) # farfield link

if not os.path.exists(os.path.join(data_dir, 'farfield_data/')):
  os.mkdir(os.path.join(data_dir, 'farfield_data/'))

if not os.path.exists(os.path.join(data_dir, 'farfield_data/train')):
  with tarfile.open(train_tar_path) as tar:
    print(f"Extracting train...")
    tar.extractall(path=os.path.join(data_dir, 'farfield_data'))


Downloaded at: /content/tar/farfield.tar
Extracting train...


In [9]:
! ls /drive/MyDrive/nis_data/farfield_test/files/ | head -n 2
! echo "-------"
! ls /content/farfield_data/train/farfield | head -n 2

ls: cannot access '/drive/MyDrive/nis_data/farfield_test/files/': No such file or directory
-------
000140f4107fabaf6e5b58520b13a6f2.wav
0001795eae39554a7167da2b9d495a28.wav


### Создаем .jsonl манифесты с абсолютными путями и сохраняем их в Гугл драйв

In [15]:
# функция для построения .jsonl файлов с абсолютными путями
def build_abs_manifest(json_path, filename_in, filename_out, data_abs_path, rel_path_starts_with):
  path_in = os.path.join(json_path, filename_in)
  path_out = os.path.join(json_path, filename_out)
  with open(path_in, 'r') as fin:
    with open(path_out, 'w+') as fout:
      for line in fin:
        metadata = json.loads(line)
        if metadata['audio_filepath'].startswith(rel_path_starts_with): # ищем пути, которые начинаются с заданного значения
          filename = metadata['audio_filepath'].replace(rel_path_starts_with, '')  # если нашли, то очищаем путь от заданного значения, чтобы получить "голое" название файла
          metadata['audio_filepath'] = os.path.join(data_abs_path, filename) # заменяем путь к файлу на абсолютный
         #print(metadata['audio_filepath'])
          json.dump(metadata, fout)
          fout.write('\n')
  return path_out
  
test_manifest_abs_path = build_abs_manifest(json_path='/drive/MyDrive/nis_data/farfield_test',
                                            filename_in='manifest.jsonl',
                                            filename_out='manifest_abs.jsonl', 
                                            data_abs_path=test_data_abs_path,
                                            rel_path_starts_with='files/'
                                            ) # манифесты с абслютными путями для farfield_test

train_manifest_abs_path = build_abs_manifest(json_path='/drive/MyDrive/nis_data/farfield_train',
                                            filename_in='manifest.jsonl',
                                            filename_out='manifest_abs.jsonl', 
                                            data_abs_path=train_data_abs_path,
                                            rel_path_starts_with='farfield/'
                                            ) # манифесты с абслютными путями для farfield_train

print(test_manifest_abs_path)
print(train_manifest_abs_path)

/drive/MyDrive/nis_data/farfield_test/manifest_abs.jsonl
/drive/MyDrive/nis_data/farfield_train/manifest_abs.jsonl


In [16]:
! echo "Test:"
! head -n 1 /drive/MyDrive/nis_data/farfield_test/manifest.jsonl
! echo 
! head -n 1 /drive/MyDrive/nis_data/farfield_test/manifest_abs.jsonl
! echo "-------"
! echo "Train:"
! head -n 1 /drive/MyDrive/nis_data/farfield_train/manifest.jsonl
! echo 
! head -n 1 /drive/MyDrive/nis_data/farfield_train/manifest_abs.jsonl

Test:
{"id": "58b586f67f5e634506e215df5996b82e", "audio_filepath": "files/58b586f67f5e634506e215df5996b82e.wav", "text": "\u0434\u0436\u043e\u0439 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.696625}

{"id": "58b586f67f5e634506e215df5996b82e", "audio_filepath": "/drive/MyDrive/nis_data/farfield_test/files/58b586f67f5e634506e215df5996b82e.wav", "text": "\u0434\u0436\u043e\u0439 \u0445\u0432\u0430\u0442\u0438\u0442", "duration": 1.696625}
-------
Train:
{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0445\u043e\u0442\u0435\u043b\u0430 \u043d\u043e\u043c\u0435\u0440 \u0442\u0435\u043b\u0435\u0444\u043e\u043d\u0430 \u043f\u043e\u043c\u0435\u043d\u044f\u0442\u044c \u043a\u0430\u0440\u0442\u044b", "duration": 5.4596875}

{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "/content/farfield_data/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0

## Установка NeMo

In [17]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2021-12-01 18:12:28 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Загрузка предобученной модели


In [ ]:
# asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_ru_quartznet15x5")

load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo", data_dir)
base_asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo")) # модель с весами

## Использование модели для распознавания

In [19]:
! ls /drive/MyDrive/nis_data/farfield_test/files/ | head -n 1
! echo "-------"
! ls /content/farfield_data/train/farfield | head -n 1

0043592661b915d774bc17e89fd695de.wav
-------
000140f4107fabaf6e5b58520b13a6f2.wav


In [20]:
files = [os.path.join(test_data_abs_path, '0043592661b915d774bc17e89fd695de.wav'),
         os.path.join(train_data_abs_path, '000140f4107fabaf6e5b58520b13a6f2.wav'),
         '/drive/MyDrive/nis_data/farfield_test/files/58b586f67f5e634506e215df5996b82e.wav']

transcriptions = base_asr_model.transcribe(paths2audio_files=files)

for fname, transcription in zip(files, transcriptions):
  print(f"Audio in {fname} was recognized as: {transcription}")

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-12-01 18:12:58 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-12-01 18:12:58 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


Audio in /drive/MyDrive/nis_data/farfield_test/files/0043592661b915d774bc17e89fd695de.wav was recognized as: салют сколько предприятий зявляются крезентами сбера
Audio in /content/farfield_data/train/farfield/000140f4107fabaf6e5b58520b13a6f2.wav was recognized as: какая будет погода завтра утром
Audio in /drive/MyDrive/nis_data/farfield_test/files/58b586f67f5e634506e215df5996b82e.wav was recognized as: джой хватит


## Расчет метрики WER для базовой модели до обучения


In [21]:
def calc_wer(model):
  # Bigger batch-size = bigger throughput
  # params['model']['validation_ds']['batch_size'] = 16
  # Setup the test data loader and make sure the model is on GPU
  # asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
  model.cuda()

  # We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
  # WER is computed as numerator/denominator.
  # We'll gather all the test batches' numerators and denominators.
  wer_nums = []
  wer_denoms = []

  # Loop over all test batches.
  # Iterating over the model's `test_dataloader` will give us:
  # (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
  # See the AudioToCharDataset for more details.
  for test_batch in model.test_dataloader():
          test_batch = [x.cuda() for x in test_batch]
          targets = test_batch[2]
          targets_lengths = test_batch[3]        
          log_probs, encoded_len, greedy_predictions = model(
              input_signal=test_batch[0], input_signal_length=test_batch[1]
          )
          # Notice the model has a helper object to compute WER
          model._wer.update(greedy_predictions, targets, targets_lengths)
          _, wer_num, wer_denom = model._wer.compute()
          model._wer.reset()
          wer_nums.append(wer_num.detach().cpu().numpy())
          wer_denoms.append(wer_denom.detach().cpu().numpy())

          # Release tensors from GPU memory
          del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

  # We need to sum all numerators and denominators first. Then divide.

  wer = sum(wer_nums)/sum(wer_denoms)
  print(f"WER модели = {wer}")
  return wer

Конфигурация базовой модели

In [22]:
from omegaconf import DictConfig

params_model = base_asr_model._cfg

params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = train_manifest_abs_path
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest_abs_path
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest_abs_path
params_model['test_ds']['batch_size']=16
params_model['optim']['lr'] = 0.05

base_asr_model.setup_optimization(optim_config=DictConfig(params_model['optim']))
base_asr_model.setup_training_data(train_data_config=params_model['train_ds'])
base_asr_model.setup_test_data(test_data_config=params_model['test_ds'])
base_asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])


[NeMo W 2021-12-01 18:12:59 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-12-01 18:12:59 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )


[NeMo W 2021-12-01 18:12:59 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2021-12-01 18:13:08 collections:173] Dataset loaded with 123995 files totalling 132.43 hours
[NeMo I 2021-12-01 18:13:08 collections:174] 8 files were filtered totalling 0.03 hours


[NeMo W 2021-12-01 18:13:08 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-12-01 18:13:09 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-12-01 18:13:09 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-12-01 18:13:09 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-12-01 18:13:09 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
calc_wer(base_asr_model)

[NeMo W 2021-12-01 14:19:36 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER модели = 0.663803388212837


0.663803388212837

## Чистка данных

Вычислим расстояния и отношения Левенштейна для каждой пары текст-предсказанный текст. Уберем те записи, которые базовая сеть распознала идеально (расстояние=0, отношение=1)

In [ ]:
# unfinished
import pandas as pd

audio_filepaths = []
true_texts = []
predicted_texts = []
json_data = []
count = 0

with open(train_manifest_abs_path, 'r') as fin: # train
  for line in fin:
    count += 1
    if count != 12367:
      input_metadata = json.loads(line)
      audio_filepaths.append(input_metadata['audio_filepath'])
      true_texts.append(input_metadata['text'])
      json_data.append(input_metadata)

predicted_texts = base_asr_model.transcribe(paths2audio_files=audio_filepaths)


In [ ]:
lv_dists = []
lv_ratios = []

for i in range(len(predicted_texts)):
  lv_dists.append(lv_dist(true_texts[i], predicted_texts[i]))
  lv_ratios.append(lv_ratio(true_texts[i], predicted_texts[i]))

data_dic = {"Levenshtein distance": lv_dists,
            "Levenshtein ratio": lv_ratios,
            "true text": true_texts,
            "predicted text": predicted_texts, 
            "json": json_data
            }
data = pd.DataFrame(data_dic)

In [ ]:
data[(data['Levenshtein ratio'] < 0.95) & (data['Levenshtein ratio'] > 0.1)].sort_values(by=['Levenshtein ratio'])

,Levenshtein distance,Levenshtein ratio,true text,predicted text,json
214,17,0.105263,кто придумал лампы,,"{'id': '9cb1ddfff3548e93697c6c9128a9e758', 'au..."
499,48,0.111111,сбер как я могу узнать тарифы за обслуживание ...,реа,"{'id': '25d4ffa19eadcbae631eb4c42b660787', 'au..."
262,21,0.153846,салют выйти из сценария,лим,"{'id': '44ed0337886cddb41cf995923acc1aca', 'au..."
248,14,0.222222,салют остановить,а,"{'id': '390ed807a936a72375677088533ed401', 'au..."
907,71,0.314050,афина здравствуйте могу ли я разблокировать св...,бле надевдх хроче деления гораза,"{'id': '7212cfa3f3339cf8ca710c89023c7fb2', 'au..."
...,...,...,...,...,...
1877,3,0.949153,джой заверши данный видеозвонок,о заверши данный видеозвонок,"{'id': '2c5c512e8ecd103424fcf903d7b100cf', 'au..."
933,2,0.949153,салют почтовый индекс батурин,салют почтовый ен декс батурин,"{'id': '0a44b030ecfcdae86b5849209e3cb457', 'au..."
957,2,0.949153,сбер включай каналы конный мир,сбер включи каналы конный мир,"{'id': 'a6f79e68bf27956d23b36efc67359168', 'au..."
1074,3,0.949495,афина сделай звонок валентине викторовне карпенко,афина сделай звонок валентине викторовне картр...,"{'id': 'c9b75c4492062b7c164a51bd5614c769', 'au..."


In [ ]:
with open(path_out, 'w+') as fout:
  json.dump(metadata, fout)
  fout.write('\n')

## Дообучение с помощью PyTorch Lightning


Конфигурация первой модели, которую будем дообучать (без чистки данных) 


In [23]:
import pytorch_lightning as pl
#trainer = pl.Trainer(gpus=1, max_epochs=1, precision=16)
trainer = pl.Trainer(gpus=1, max_epochs=1, precision=16) 
trained_asr_model = base_asr_model

# trained_asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

params_model = trained_asr_model._cfg

params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = train_manifest_abs_path
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = test_manifest_abs_path
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = test_manifest_abs_path
params_model['test_ds']['batch_size']=16

params_model['optim']['lr'] = 0.0001

trained_asr_model.setup_optimization(optim_config=DictConfig(params_model['optim']))
trained_asr_model.setup_training_data(train_data_config=params_model['train_ds'])
trained_asr_model.setup_test_data(test_data_config=params_model['test_ds'])
trained_asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
[NeMo W 2021-12-01 18:13:10 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-12-01 18:13:10 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )


[NeMo W 2021-12-01 18:13:10 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2021-12-01 18:13:14 collections:173] Dataset loaded with 123995 files totalling 132.43 hours
[NeMo I 2021-12-01 18:13:14 collections:174] 8 files were filtered totalling 0.03 hours


[NeMo W 2021-12-01 18:13:14 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-12-01 18:13:14 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-12-01 18:13:14 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-12-01 18:13:14 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-12-01 18:13:14 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
# files = [os.path.join(test_data_abs_path, '0043592661b915d774bc17e89fd695de.wav'),
#          os.path.join(train_data_abs_path, '000140f4107fabaf6e5b58520b13a6f2.wav'),
#          '/drive/MyDrive/nis_data/farfield_test/files/58b586f67f5e634506e215df5996b82e.wav']

# trained_asr_model.transcribe(paths2audio_files=files)


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-11-30 14:34:49 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


['салют сколько предприятий зявляются крезентами сбера',
 'какая будет погода завтра утром',
 'джой хватит']

Начинаем обучение

In [24]:
trainer.fit(trained_asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-12-01 18:13:14 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-12-01 18:13:14 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )


[NeMo W 2021-12-01 18:13:14 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-12-01 18:13:15 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo E 2021-12-01 18:13:22 segment:156] Loading /drive/MyDrive/nis_data/farfield_test/files/5463033113ec14d87ac2e357ef1b8dcb.wav via SoundFile raised RuntimeError: `Error opening '/drive/MyDrive/nis_data/farfield_test/files/5463033113ec14d87ac2e357ef1b8dcb.wav': System error.`. NeMo will fallback to loading via pydub.
[NeMo W 2021-12-01 18:13:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is 

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [25]:
calc_wer(trained_asr_model) 

[NeMo W 2021-12-01 22:05:02 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER модели = 0.5329277022190408


0.5329277022190408

In [26]:
trained_asr_model.save_to('/drive/MyDrive/nis_data/first_trained_model.nemo')